In [1]:
import os
import datetime as dt

import numpy as np
import pandas as pd
import tensorflow as tf

from constants import NP_SEED, TF_SEED
np.random.seed(NP_SEED)
tf.set_random_seed(TF_SEED)

from nn_clf import CNN_to_RNN, Dense, RAND_CNN, RNN
import etl
import util
from visualizer import plot_metrics
from ensembler import Ensembler

N_CLASS = 3
N_LOOKBACK = 8
SPLIT_RATIO = (0.7, 0.15, 0.15)  # train, val, test

# Step1. Data Extract, Transform and Load
랜덤 시드 변경을 적용하려면, reload=True로 설정해야 함

In [2]:
%%time
etler = etl.ETL(N_LOOKBACK, SPLIT_RATIO, downsampling=False, reload=False)

# val과 test 데이터에서 swell_t-1과 맞춰야하는 swell이 다른 샘플 개수 출력
print((etler.ds['val']['x'][:, -1, -1] != etler.ds['val']['y'][:, -1]).sum())
print((etler.ds['test']['x'][:, -1, -1] != etler.ds['test']['y'][:, -1]).sum())

# train 데이터에서 swell_t-1과 맞춰야하는 swell이 다른 샘플들을 오버샘플링
etler.oversampling(3)

Load dataset from existing dataset


/home/yongwoo/anaconda3/envs/ipykernel_py36/lib/python3.6/site-packages/IPython/core/magics/execution.py:1238: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code, glob, local_ns)



  train   x (4616, 8, 249)
  train   y (4616, 1)
  train  dt (4616,)
    val   x (0,)
    val   y (0, 1)
    val  dt (0,)
   test   x (0,)
   test   y (0, 1)
   test  dt (0,)
problem   x (600, 8, 249)
problem  dt (600,)

[train data 분포]
0 : 1 : 2 = 0.45 : 0.55 : 0.0
[val data 분포]
0 : 1 : 2 = nan : nan : nan
[test data 분포]
0 : 1 : 2 = nan : nan : nan


/home/yongwoo/AI-Hackerton/Posco-ai-challnege/0830_last/etl.py:54: RuntimeWarning: invalid value encountered in long_scalars
  print('0 : 1 : 2 = {:.2} : {:.2} : {:.2}'.format((d0_dict['y'] == 0).sum() / len(d0_dict['y']),
/home/yongwoo/AI-Hackerton/Posco-ai-challnege/0830_last/etl.py:55: RuntimeWarning: invalid value encountered in long_scalars
  (d0_dict['y'] == 1).sum() / len(d0_dict['y']),
/home/yongwoo/AI-Hackerton/Posco-ai-challnege/0830_last/etl.py:56: RuntimeWarning: invalid value encountered in long_scalars
  (d0_dict['y'] == 2).sum() / len(d0_dict['y'])))


IndexError: too many indices for array

# Step2. Build Neural Network Models

In [ ]:
# tf 그래프 초기화 및 세션 생성
tf.reset_default_graph()
sess = tf.Session()
BATCH_SIZE = 64
N_FEATURE = etler.ds['train']['x'].shape[-1]
N_ENSEMBLE = 1
EPOCH = 30

In [ ]:
nn_list = []  # 앙상블 할 모델들의 리스트
predicts_list = [] # 각 모델의 predicts(val,test,problem에 대한 epoch당 예측결과)의 리스트
# for i in range(N_ENSEMBLE): # 앙상블 할 모델들의 그래프 생성(build)
#     nn = Dense(sess, N_CLASS, N_LOOKBACK, N_FEATURE, 
#                dropout_keep_prob=0.5,
#                lr=1e-4,
#                lr_decay=0.90,
# #                feature_size_list=etler.feature_size_list,
#                name='dense'+str(i))
#     nn_list.append(nn)
    
for i in range(N_ENSEMBLE): # 앙상블 할 모델들의 그래프 생성(build)
    nn = RAND_CNN(sess, N_CLASS, N_LOOKBACK, N_FEATURE, 
               dropout_keep_prob=0.5,
               lr=1e-4,
               lr_decay=0.90,
#                feature_size_list=etler.feature_size_list,
               name='CNN'+str(i))
    nn_list.append(nn)
sess.run(tf.global_variables_initializer())

# Step3. Model Ensemble and Save the Result

In [ ]:
ensembler = Ensembler(etler.ds, BATCH_SIZE, EPOCH, 
                        feature_shuffle=True, # swell_t-1를 제외한 피쳐 셔플 여부
                        train_all_data=False, # val, test 데이터를 학습에 사용할 지 여부
                        nn_list=nn_list
                     )

In [ ]:

ensembler.train_per_nn()  # nn 하나 EPOCH 학습 후, 다음 nn 학습


# Step X. 한 모델만 돌려보는 경우 아래 코드 이용

In [ ]:
# tf 그래프 초기화 및 세션 생성
tf.reset_default_graph()
sess = tf.Session()
BATCH_SIZE = 64
N_FEATURE = etler.ds['train']['x'].shape[-1]
N_ENSEMBLE = 3
EPOCH = 30
#RAND_CNN
nn = CNN_to_RNN(sess, N_CLASS, N_LOOKBACK, N_FEATURE, 
           dropout_keep_prob=0.5,
           lr=3e-5,
           lr_decay=0.95,
           feature_size_list=etler.feature_size_list,
           name='nn')

sess.run(tf.global_variables_initializer())
predicts = nn.train(etler.ds, BATCH_SIZE, EPOCH, 
                    feature_shuffle=False, # swell_t-1를 제외한 피쳐 셔플 여부
                    train_all_data=False, # val, test 데이터를 학습에 사용할 지 여부
                    verbose=True
                    )

#util.save_result_excel(predicts['problem'][-1])

In [ ]:
tf.